In [8]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv("df_merged_cleanfinal.csv")

# Filtrar las filas donde "sentiment_analysis" no es igual a 1.0
df = df[df['sentiment_analysis'] != 1.0]

# Guardar el DataFrame resultante en un nuevo archivo CSV
df.to_csv("df_merged_cleanfinal_filtered.csv", index=False)


In [9]:
import pandas as pd

# Cargar el archivo CSV filtrado
df_filtered = pd.read_csv("df_merged_cleanfinal_filtered.csv")

# Contar el número de filas
num_filas = len(df_filtered)

print(f"Número de filas restantes: {num_filas}")


Número de filas restantes: 14565


In [10]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('df_merged_cleanfinal_filtered.csv')

# Reemplazar los valores 0.0 por 0 y 2.0 por 2
df['sentiment_analysis'] = df['sentiment_analysis'].replace({0.0: 0, 2.0: 2})

# Convertir la columna 'sentiment_analysis' a tipo int
df['sentiment_analysis'] = df['sentiment_analysis'].astype(int)

# Guardar el DataFrame de nuevo en un archivo CSV si es necesario
df.to_csv('df_merged_cleanfinal_filtered.csv', index=False)


In [11]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('df_merged_cleanfinal_filtered.csv')

# Consultar las primeras filas de cada columna
print(df.head())


   genres release_date    id            user_id  playtime_forever  \
0  Action   2000-11-01    10  76561197970982479                 6   
1  Action   2006-03-14  1200          evcentric               923   
2  Action   2010-07-12   300              doctr              1131   
3  Action   2004-11-01   240          maplemage              5751   
4  Action   2004-11-01   240        MeaTCompany              3209   

                         title  sentiment_analysis  
0                CounterStrike                   2  
1  Red Orchestra Ostfront 4145                   2  
2         Day of Defeat Source                   2  
3         CounterStrike Source                   2  
4         CounterStrike Source                   0  


In [ ]:
#Codigo que utiliza sentiment_analysis y cubre: def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [31]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar los datos desde el archivo CSV
df = pd.read_csv('df_merged_cleanfinal_filtered.csv')

# Convertir la columna 'sentiment_analysis' a cadena
df['sentiment_analysis'] = df['sentiment_analysis'].astype(str)

# Crear una nueva columna 'content' que combine 'genres' y 'sentiment_analysis'
df['content'] = df[['genres', 'sentiment_analysis']].apply(lambda x: ' '.join(x), axis=1)

# Crear una instancia de TfidfVectorizer para transformar el texto a vectores TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Construir la matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(df['content'])

# Crear un modelo KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine')  # Cambia la métrica a 'cosine'

# Entrenar el modelo KNN con los datos
knn_model.fit(tfidf_matrix)

# Función para obtener 5 recomendaciones únicas
def get_unique_recommendations(game_id):
    game_index = df[df['id'] == game_id].index[0]
    distances, indices = knn_model.kneighbors(tfidf_matrix[game_index])
    recommended_games = []
    recommended_set = set()
    for i in range(1, len(distances.flatten())):
        game = df.iloc[indices.flatten()[i]]
        game_id = game['id']
        if game_id not in recommended_set:
            recommended_set.add(game_id)
            recommended_games.append((game['title'], game['sentiment_analysis']))
        if len(recommended_games) >= 5:
            break
    return recommended_games

# Ejemplo de cómo obtener 5 recomendaciones únicas para un juego con ID 15100
game_id = 15100
recommendations = get_unique_recommendations(game_id)
print(f"Recomendaciones para el juego con ID {game_id}:")
for i, (game, sentiment) in enumerate(recommendations):
    sentiment_label = 'Malo' if sentiment == '0' else 'Positivo'
    print(f"{i + 1}. {game} - Sentimiento: {sentiment_label}")



Recomendaciones para el juego con ID 15100:
1. Batman Arkham Asylum Game of the Year Edition - Sentimiento: Positivo
2. Portal 2 - Sentimiento: Positivo
3. LEGO Batman - Sentimiento: Positivo
4. Just Cause 2 - Sentimiento: Malo


In [ ]:
#Codigo que utiliza NO utiliza sentiment_analysis y cubre: def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [49]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar los datos desde el archivo CSV
df = pd.read_csv('df_merged_cleanfinal.csv')

# Crear una instancia de TfidfVectorizer para transformar el texto a vectores TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Construir la matriz TF-IDF con la columna 'genres'
tfidf_matrix = tfidf.fit_transform(df['genres'])

# Crear un modelo KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine')  # Cambia la métrica a 'cosine'

# Entrenar el modelo KNN con los datos
knn_model.fit(tfidf_matrix)

# Función para obtener 5 recomendaciones únicas
def get_unique_recommendations(game_id):
    game_index = df[df['id'] == game_id].index[0]
    distances, indices = knn_model.kneighbors(tfidf_matrix[game_index])
    recommended_games = []
    recommended_set = set()
    for i in range(1, len(distances.flatten())):
        game = df.iloc[indices.flatten()[i]]
        game_id = game['id']
        if game_id not in recommended_set:
            recommended_set.add(game_id)
            recommended_games.append(game['title'])
        if len(recommended_games) >= 5:
            break
    return recommended_games

# Ejemplo de cómo obtener 5 recomendaciones únicas para un juego con ID 15100
game_id = 15100
recommendations = get_unique_recommendations(game_id)
print(f"Recomendaciones para el juego con ID {game_id}:")
for i, game in enumerate(recommendations):
    print(f"{i + 1}. {game}")


Recomendaciones para el juego con ID 15100:
1. Mirrors Edge
2. Just Cause 2
3. Portal 2


In [54]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
import numpy as np

# Cargar los datos desde el archivo CSV
df = pd.read_csv('df_merged_cleanfinal.csv')

# Crear una instancia de TfidfVectorizer para transformar el texto a vectores TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Construir la matriz TF-IDF con la columna 'genres'
tfidf_matrix = tfidf.fit_transform(df['genres'])

# Crear un modelo KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine')

# Dividir los datos en K conjuntos (K-fold)
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Puedes ajustar el número de splits según tus necesidades

# Inicializar una lista para almacenar las métricas de rendimiento
performance_metrics = []

for train_index, test_index in kf.split(df):
    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test = tfidf_matrix[train_index], tfidf_matrix[test_index]

    # Entrenar el modelo KNN con los datos de entrenamiento
    knn_model.fit(X_train)

    # Evaluar el modelo en los datos de prueba
    avg_recall = 0
    for game_id in df.iloc[test_index]['id']:
        recommendations = get_unique_recommendations(game_id)
        true_positives = len(set(recommendations).intersection(set(df[df['id'] == game_id]['title'])))
        recall = true_positives / 5  # Calcular el recall
        avg_recall += recall

    avg_recall /= len(test_index)  # Promedio del recall para este fold
    performance_metrics.append(avg_recall)

# Calcular el promedio de los recalls en los K-folds
mean_recall = np.mean(performance_metrics)

print(f"Promedio del Recall en {kf.get_n_splits()} folds: {mean_recall}")


Promedio del Recall en 5 folds: 0.07174960298899337


In [63]:
import pandas as pd

# Carga el archivo CSV en un DataFrame
df = pd.read_csv('df_merged_cleanfinal.csv')

# Encuentra el género del juego con ID 4000
game_genre = df[df['id'] == 4000]['genres'].values[0]

# Encuentra los IDs de los juegos que tienen el mismo género
same_genre_ids = df[df['genres'] == game_genre]['id'].unique()

print(f"Los IDs de los juegos que tienen el mismo género que el juego con ID 4000 son: {same_genre_ids}")


Los IDs de los juegos que tienen el mismo género que el juego con ID 4000 son: [  4000  92800 227300 232010 258760 319510 220200  70110]


In [1]:
import pandas as pd

# Carga el archivo CSV en un DataFrame
df = pd.read_csv('df_merged_cleanfinal.csv')

# Lista de IDs de juegos
game_ids = [4000, 92800, 227300, 232010, 258760, 319510, 220200, 70110]

# Crea un diccionario donde las claves son los IDs de los juegos y los valores son listas de los IDs de usuario que han jugado estos juegos
game_users = {game_id: list(df[df['id'] == game_id]['user_id'].unique()) for game_id in game_ids}

for game_id, user_ids in game_users.items():
    print(f"Los usuarios que han jugado el juego con ID {game_id} son: {user_ids}")

Los usuarios que han jugado el juego con ID 4000 son: ['corrupted_soul', '76561198089393905', '76561198156664158', '76561198077246154', 'DJKamBer', 'Rainbow-Dashie', 'btuty8trgt', 'fdcfelipefdc', 'bbaekhyun', 'ABSOLUTLY_NOTHING', 'devvonst', '76561198048353577', '76561198066046412', 'tru_mu_', '76561198070565427', 'SavageEeEe', 'Leaf_Light_Moscow', '76561198085547365', '76561198056147391', '8Bit-Ninja', '76561198071500702', '76561198047407223', 'latiano', '76561198100297613', '76561198094630961', '76561198092411867', 'diego9031', 'iampiepiepie', '76561198079863506', '76561198078738103', '76561198067936203', '76561198079770573', 'UpTheAss', '76561198075491015', 'Derp-e', '76561198138053270', '76561198062422084', 'Life-Of-Blu', 'maxgp', 'AppaTheSwagBison', 'knightboy991', '76561198034678103', 'TheDropix101', '76561198093817084', '76561198102421718', '76561198087407319', 'damo131', '19702316748', 'Coldaniel', 'Berserklejerker', 'possessioniskey', '76561198072732874', '76561198093112529', 

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Crear un diccionario con las recomendaciones generadas por tu modelo
recommendations = {
    15100: ['Mirrors Edge', 'Just Cause 2', 'Portal 2'],
    # Agregar más recomendaciones para otros juegos si es necesario
}

# Crear un diccionario con las preferencias reales de los usuarios en el conjunto de prueba
ground_truth = {
    '76561198079334894': ['232010'],  
    '76561198081521900': ['232010'],
    '76561198103139400': ['258760'],
    '76561198159513700': ['319510'],
    '76561198095035800': ['319510'],
    '9990008888': ['319510'],
    '76561198091509600': ['220200'],
    'eyyrabb1t': ['220200'],  
    '76561198065175700': ['220200'],
    '76561198073770000': ['220200'],
    'meshcollider': ['70110'],
    # Juegos relevantes para otros usuarios
}

# Crear listas para almacenar los resultados de Recall y Precision
recalls = []
precisions = []

# Calcular Recall y Precision para cada juego en el conjunto de prueba
for user_id, relevant_games in ground_truth.items():
    if user_id in recommendations:
        recommended_games = recommendations[user_id]
        
        # Calcular Recall y Precision para este usuario
        true_positives = len(set(relevant_games).intersection(recommended_games))
        recall = true_positives / len(relevant_games) if len(relevant_games) > 0 else 0.0
        precision = true_positives / len(recommended_games) if len(recommended_games) > 0 else 0.0
        
        recalls.append(recall)
        precisions.append(precision)

# Calcular el promedio de Recall y Precision
mean_recall = sum(recalls) / len(recalls) if len(recalls) > 0 else 0.0
mean_precision = sum(precisions) / len(precisions) if len(precisions) > 0 else 0.0

# Calcular el F1-Score
f1_score = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0.0

# Calcular el MAP (Mean Average Precision)
average_precision = []
for user_id, relevant_games in ground_truth.items():
    if user_id in recommendations:
        recommended_games = recommendations[user_id]
        precision_at_k = []
        relevant_count = 0
        for i, game in enumerate(recommended_games):
            if game in relevant_games:
                relevant_count += 1
                precision_at_k.append(relevant_count / (i + 1))
        if not precision_at_k:
            precision_at_k.append(0)
        average_precision.append(sum(precision_at_k) / len(precision_at_k))

map_score = sum(average_precision) / len(average_precision) if len(average_precision) > 0 else 0.0

print(f"Recall: {mean_recall}")
print(f"Precision: {mean_precision}")
print(f"F1-Score: {f1_score}")
print(f"Mean Average Precision (MAP): {map_score}")



Recall: 0.0
Precision: 0.0
F1-Score: 0.0
Mean Average Precision (MAP): 0.0


In [15]:
import pandas as pd

# Cargar los datos desde el archivo CSV
df = pd.read_csv('df_merged_cleanfinal.csv')

# Ajustar la columna 'genres' para que solo contenga el primer género
df['genres'] = df['genres'].apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)

# Guardar el DataFrame ajustado en un nuevo archivo CSV
df.to_csv('df_merged_cleanfinalA.csv', index=False)


In [16]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Cargar los datos desde el archivo CSV
df = pd.read_csv('df_merged_cleanfinalA.csv')

# Crear una instancia de TfidfVectorizer para transformar el texto a vectores TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Crear un modelo KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine')

# Crear un diccionario con las preferencias reales de los usuarios en el conjunto de prueba
ground_truth = {
    '76561198056450930': ['620'],  
    '76561198100073188': ['8190'],
    '76561198306855779': ['12200'],
    '76561198085294887': ['12790'],
    'Buggark': ['620'],
    '76561198057772341': ['17410'],
    '76561198038712243': ['8190'],
     # Juegos relevantes para otros usuarios
}

def get_unique_recommendations(game_id, ground_truth):
    # Obtener el género del juego con el ID especificado
    genres = df[df['id'] == game_id]['genres'].values[0].split(', ')
    
    # Filtrar el DataFrame para incluir solo juegos con al menos uno de los géneros del juego de referencia
    filtered_df = df[df['genres'].str.split(', ').apply(lambda x: any(item in genres for item in x))]
    
    # Construir la matriz TF-IDF con la columna 'genres' de los juegos filtrados
    tfidf_matrix = tfidf.fit_transform(filtered_df['genres'].apply(lambda x: ' '.join(x.split(', '))))
    
    # Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
    X_train, X_test = train_test_split(tfidf_matrix, test_size=0.2, random_state=42)
    
    # Entrenar el modelo KNN con los datos de entrenamiento
    knn_model.fit(X_train)

    game_index = filtered_df[filtered_df['id'] == game_id].index[0] if len(filtered_df[filtered_df['id'] == game_id].index) > 0 else None

    if game_index is not None:
        distances, indices = knn_model.kneighbors(tfidf_matrix[game_index])
        recommended_games = []
        recommended_set = set()
        for i in range(1, len(distances.flatten())):
            game = filtered_df.iloc[indices.flatten()[i]]
            game_id = game['id']
            if game_id not in recommended_set:
                recommended_set.add(game_id)
                recommended_games.append(game['title'])
            if len(recommended_games) >= 5:
                break
        
        # Compara las recomendaciones con las preferencias reales del usuario
        user_id = filtered_df.iloc[game_index]['user_id']
        real_preferences = ground_truth.get(user_id, [])
        relevancia = sum(1 for juego in recommended_games if juego in real_preferences)
        return recommended_games, relevancia
    else:
        return [], 0

# Ejemplo de cómo obtener 5 recomendaciones únicas para un juego con ID 15100 en el conjunto de prueba
game_id = 15100
recommendations, relevancia = get_unique_recommendations(game_id, ground_truth)
print(f"Recomendaciones para el juego con ID {game_id}:")
for i, game in enumerate(recommendations):
    print(f"{i + 1}. {game}")
print(f"Relevancia: {relevancia} / 5")



Recomendaciones para el juego con ID 15100:
1. CounterStrike Source
2. Call of Duty Modern Warfare 2
3. HalfLife 2
4. Dino DDay
Relevancia: 0 / 5
